In [1]:
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import torch
from torch.utils.data import DataLoader

from transformers import GPT2Config, GPT2Tokenizer, BertModel, BertTokenizer, DistilBertModel, DistilBertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

from InductiveAttentionModels import GPT2InductiveAttentionHeadModel
from loss import SequenceCrossEntropyLoss

from trainer import Trainer
import time
import tqdm
from dataset import MovieRecDataset, RecDataset
from corrected_mese import C_UniversalCRSModel
from corrected_engine import C_Engine
from utilities import get_memory_free_MiB
from metrics import distinct_metrics, bleu_calc_all


In [2]:
bert_tokenizer = DistilBertTokenizer.from_pretrained("../../../../offline_transformers/distilbert-base-uncased/tokenizer")
bert_model_recall = DistilBertModel.from_pretrained('../../../../offline_transformers/distilbert-base-uncased/model')
bert_model_rerank = DistilBertModel.from_pretrained('../../../../offline_transformers/distilbert-base-uncased/model')
gpt_tokenizer = GPT2Tokenizer.from_pretrained("../../../../offline_transformers/gpt2/tokenizer")
gpt2_model = GPT2InductiveAttentionHeadModel.from_pretrained('../../../../offline_transformers/gpt2/model')


In [3]:
REC_TOKEN = "[REC]"
REC_END_TOKEN = "[REC_END]"
SEP_TOKEN = "[SEP]"
PLACEHOLDER_TOKEN = "[MOVIE_ID]"
gpt_tokenizer.add_tokens([REC_TOKEN, REC_END_TOKEN, SEP_TOKEN, PLACEHOLDER_TOKEN])
gpt2_model.resize_token_embeddings(len(gpt_tokenizer)) 

test_path = "data/processed/durecdial2_all_dev_placeholder_updated"
items_db_path = "data/processed/durecdial2_full_entity_db_placeholder"
output_file_path = 'out/retrieval_results_best.txt'

items_db = torch.load(items_db_path)

test_dataset = RecDataset(torch.load(test_path), bert_tokenizer, gpt_tokenizer)


In [7]:
test_dataset[0][1][0][0].shape

torch.Size([1, 14])